In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from pipeline import *
from models import *

In [7]:
pipeline = AABB245_Pipeline(num_blocks=5, extrema_th=1)
dataset = PolymerDataset(['../data/AA66266AA.npy', '../data/AA66466AA.npy'], pipeline)

Processing ../data/AA66466AA.npy: 100%|██████████| 20752/20752 [01:52<00:00, 184.37it/s]


In [8]:
dataset.data.shape, dataset.labels.shape

(torch.Size([40837, 5, 19]), torch.Size([40837]))

In [9]:
train_data, test_data = train_test_split(dataset)

In [10]:
model = VanillaLSTM(input_dim=dataset.num_features, output_dim=dataset.num_classes)

In [11]:
model = train(train_data, model, batch_size=512, num_epochs=200, lr_rate=0.01)

epoch=0/200, loss=0.4219734715297818, accuracy=79.18579864501953
epoch=1/200, loss=0.33192903781309724, accuracy=84.50260162353516
epoch=2/200, loss=0.31340299313887954, accuracy=85.49739837646484
epoch=3/200, loss=0.3044575727544725, accuracy=85.97183990478516
epoch=4/200, loss=0.30318702617660165, accuracy=85.94429016113281
epoch=5/200, loss=0.30155950831249356, accuracy=86.2381362915039
epoch=6/200, loss=0.30554674845188856, accuracy=86.04530334472656
epoch=7/200, loss=0.3005656749010086, accuracy=86.2105941772461
epoch=8/200, loss=0.29558743815869093, accuracy=86.26568603515625
epoch=9/200, loss=0.29784374218434095, accuracy=86.35751342773438
epoch=10/200, loss=0.297371915075928, accuracy=86.35445404052734
epoch=11/200, loss=0.2964331042021513, accuracy=86.37894439697266
epoch=12/200, loss=0.2945554235484451, accuracy=86.60238647460938
epoch=13/200, loss=0.2900005951523781, accuracy=86.79216766357422
epoch=14/200, loss=0.29059783602133393, accuracy=86.75237274169922
epoch=15/200, l

In [ ]:
len(test_data)

In [ ]:
test_loader = DataLoader(test_data)
test_X, test_y = next(iter(test_loader))

In [ ]:
test_X.shape

In [ ]:
pipeline = AA0066_Pipeline(num_blocks=3)
dataset = PolymerDataset(["../data/AA00400AA.npy", "../data/AA66466AA.npy"], pipeline)

In [ ]:
dataset.data.shape, dataset.labels.shape

In [ ]:
train_data, test_data = train_test_split(dataset)

In [ ]:
model = VanillaLSTM(input_dim=dataset.num_features, output_dim=dataset.num_classes)

In [ ]:
model = train(train_data, model, batch_size=512, num_epochs=100)